In [3]:
%matplotlib inline

GNN 활용 Link Prediction  
===========================================
이 튜토리얼은 링크 예측(link prediction), 즉 그래프에서 임의의 두 노드 사이에 에지의 존재를 예측하기 위해 GNN을 훈련시키는 방법을 알려줍니다.

Overview
------------------------
소셜 추천, 아이템 추천, 지식 그래프 완성 등과 같은 많은 애플리케이션은 두 특정 노드 사이에 에지가 존재하는지 여부를 예측하는 링크 예측으로 공식화될 수 있습니다. 이 튜토리얼은 두 논문 사이의 인용 관계(인용 또는 인용 중)가 인용 네트워크에 존재하는지 여부를 예측하는 예를 보여줍니다.
  
  
By the end of this tutorial
---------------------------

1. GNN 기반 링크 예측 모델을 구축합니다.

2. DGL 제공 데이터 세트에서 모델을 훈련하고 평가합니다.

DGL 라이브러리 설치 
-------------------------

In [4]:
!pip install dgl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 15.0 MB/s eta 0:00:00
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


필요 라이브러리 로드 
-----------------

In [5]:
# 필요 라이브러리 Load
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Overview of Link Prediction with GNN
=====================================

링크 예측 문제를 다음과 같이 이진 분류 문제로 공식화합니다.
------------------------------------

-  그래프의 에지(link)를 *positive examples*로 취급한다.
-  존재하지 않는 여러 에지(즉, 노드 사이에 에지가 없는 노드 쌍)을 *negative examples*로 취급한다.
-  *positive examples* 과 *negative examples*를 train 세트와 test 세트로 나눈다.
-  모델 평가 기준 : AUC(Area Under Curve).


<div class="alert alert-info"><h4>Note - 아직 남겨둠 </h4><p>The practice comes from
   `SEAL <https://papers.nips.cc/paper/2018/file/53f0d7c537d99b3824f0f99d62ea2428-Paper.pdf>`__,
   although the model here does not use their idea of node labeling.</p></div>





Graph and features 로드
--------------------------

Cora dataset 로드한다.

In [6]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

Extracting file to /root/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


Training and testing sets 준비
---------------------------------

This tutorial randomly picks 10% of the edges for positive examples in
the test set, and leave the rest for the training set. It then samples
the same number of edges for negative examples in both sets.

Test set에서 *positive examples*를 위해 edge의 10%를 임의로 선택하고, 나머지는 train set으로 남겨 둡니다. 그런 다음 두 set 모두에서 *negative examples*에 ​​대해 동일한 수의 edge를 샘플링합니다.

In [7]:
# Split edge set for training and testing
u, v = g.edges() # u : 시작 노드 , v : 도착 노드 (Tensor type)

eids = np.arange(g.number_of_edges()) # g.number_of_edges() : int type -> eids : ndarray type
# np.arange(g.number_of_edges()) : array([0, 1, 2, 3, 4])
eids = np.random.permutation(eids) # shuffle
# eids = array([3, 4, 1, 0, 2])

test_size = int(len(eids) * 0.1) # 1055개
train_size = g.number_of_edges() - test_size # 10556 - 1055

# u, v에서 같은 index(i)는 시작 노드(u[i])와 도착 노드(v[i]) 사이의 연결된 edge를 의미한다
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]] # test size 만큼 pair of nodes(=edges) 형태로 저장한다
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]] # train size 만큼 pair of nodes(=edges) 형태로 저장한다

# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy()))) # coo_matrix로 인접 행렬 생성한다 
# sp.coo_matrix((data, (row, col))) , coo_matrix Reference : https://radish-greens.tistory.com/1
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
# adj.todense() : sp matrix(정보 압축 상태) -> numpy matrix로 형변환 해준다.
# np.eye() : identity matrix (단위 행렬)
# adj_neg = edge가 존재하지 않으면 1, edge가 존재하면 0 (self-loop는 고려 X -> 0으로 만든다)
neg_u, neg_v = np.where(adj_neg != 0)
# edge가 존재하지 않은 값들에 대해서 negative examples 생성 

neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
# len(neg_u) 즉, negative examples는 7320000가 존재하지만 positive examples 만큼 추출하여 sample 수를 맞춰준다.

test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]] # test size 만큼 pair of nodes(=edges) 형태로 저장한다
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]] # train size 만큼 pair of nodes(=edges) 형태로 저장한다

학습할 때 원본 그래프에서 test set의 edge를 제거(=train set) 해야 한다. 
-------------------------------
~~~ 
dgl.remove_edges 
~~~  


In [ ]:
train_g = dgl.remove_edges(g, eids[:test_size])
# Train Gragh : num_nodes=2708, num_edges=9501

GraphSAGE 모델 정의  
------------------------
~~~
GraphSAGE 논문 : https://arxiv.org/abs/1706.02216
~~~

In [ ]:
from dgl.nn import SAGEConv
# dgl.nn : Pytorch nn과 동일한 라이브러리 

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean') # aggregate function 3가지(mean, full, lstm) 중 하나를 적용한다.
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

모델은 임의의 두 개의 노드 사이의 edge 존재 확률을 예측한다.

\begin{align}\hat{y}_{u\sim v} = f(h_u, h_v)\end{align}




Positive graph, negative graph and ``apply_edges``
---------------------------------------------------

- link prediction을 하기 위해서는 *pairs of nodes*를 계산해야 한다.  
- DGL 라이브러리 사용하여 *positive graph* 와 *negative graph*를 구성할 수 있다.

In [ ]:
# Graph : dgl.graph((u, v), num_nodes = g.number_of_nodes()) 만들어 준다.
# (u, v) : edge의 정보
# num_nodes = g.number_of_nodes() : 노드의 개수

# Graph 표현은 Edge List로 표현된다.
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

In [ ]:
import dgl.function as fn

# GraphSAGE에서 나오는 weight 값들에 대해서 마지막에 결과 예측 하기 위해 사용한다. 
class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h  # dictionary에 ('h' : key , h : value) 추가
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.

            print(g.edata)
            # apply_edges(function) : function의 결과 값으로 update edge feature (이때 function 은  DGL Built-in Function or a User-defined Functions.)
            # fn.u_dot_v(src, dst, key) : 기존에 key:value가 존재한다면 src,dst,key를 내적. 없다면 edge feature에 key 추가 value는 src,dst내적 값
            g.apply_edges(fn.u_dot_v('h', 'h', 'score')) # apply_edges() : new edge features를 기존의 edge features 와 node features에 기반하여 계산해 주는 함수이다.
            print(g.edata)

            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

In [ ]:
# 위 과정이 이해하기 어려운 경우 자체적으로 아래의 함수를 이용해 사용 가능하다

class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

Training Loop
-------------

전체 모델, 손실 함수(loss
function), 평가 지표를 정의할 수 있다.

- 손실 함수(loss function)는 simply binary cross entropy loss 이다.

- \begin{align}\mathcal{L} = -\sum_{u\sim v\in \mathcal{D}}\left( y_{u\sim v}\log(\hat{y}_{u\sim v}) + (1-y_{u\sim v})\log(1-\hat{y}_{u\sim v})) \right)\end{align}

- 평가 지표는 AUC를 활용한다.




In [ ]:
model = GraphSAGE(train_g.ndata['feat'].shape[1], 16)
# You can replace DotPredictor with MLPPredictor.
# pred = MLPPredictor(16)
pred = DotPredictor()


def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]) 
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]) 
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy() # 예측 값
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy() # 실제 값
    return roc_auc_score(labels, scores)

In [ ]:
# ----------- 3. set up loss and optimizer -------------- #
# in this case, loss will in training loop
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01) # 효율적인 concat을 위해 itertools.chain 사용 

# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(100):
    # forward
    h = model(train_g, train_g.ndata['feat'])
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))


{'weight': tensor([[0.0003],
        [0.0034],
        [0.0044],
        ...,
        [0.0014],
        [0.0021],
        [0.0061]], grad_fn=<GSDDMMBackward>)}
{'weight': tensor([[0.0003],
        [0.0034],
        [0.0044],
        ...,
        [0.0014],
        [0.0021],
        [0.0061]], grad_fn=<GSDDMMBackward>), 'score': tensor([[0.0005],
        [0.0056],
        [0.0014],
        ...,
        [0.0048],
        [0.0017],
        [0.0015]], grad_fn=<GSDDMMBackward>)}
{}
{'score': tensor([[7.5727e-05],
        [3.3760e-03],
        [7.9384e-04],
        ...,
        [4.4486e-03],
        [1.8336e-03],
        [3.3543e-03]], grad_fn=<GSDDMMBackward>)}
In epoch 0, loss: 0.6929933428764343
{'weight': tensor([[0.0003],
        [0.0034],
        [0.0044],
        ...,
        [0.0014],
        [0.0021],
        [0.0061]], grad_fn=<GSDDMMBackward>)}
{'weight': tensor([[0.0003],
        [0.0034],
        [0.0044],
        ...,
        [0.0014],
        [0.0021],
        [0.0061]], grad_f